In [ ]:
pip install phe

In [ ]:
import time
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px
from sympy import symbols, Poly, simplify
import random
from phe import paillier

In [ ]:
random.seed(42)

def generate_random_values(num_parties, values_per_party, num_experiments):
    random_values = {}
    for experiment in range(num_experiments):
        experiment_data = {}
        for party in range(num_parties):
            values = [random.randint(1, 1000) for _ in range(values_per_party)]
            experiment_data[party] = values
        random_values[experiment] = experiment_data
    return random_values

In [ ]:
def no_privacy_protection(num_parties, values_per_party, num_experiments, random_values):
    def generate_data(num_parties, values_per_party, random_values):
        data = {}
        for party in range(num_parties):
            data[party] = random_values[party]  # Use random values from the argument
        return data

    def compute_average(data, num_parties, values_per_party):
        total_sum = 0
        total_count = 0

        for party in range(num_parties):
            total_sum += sum(data[party])
            total_count += len(data[party])

        average = total_sum / total_count
        return average

    average_runtimes = []
    results = []

    for experiment in range(num_experiments):
        data = generate_data(num_parties, values_per_party, random_values[experiment])

        start_time = time.time()  # Record start time
        average = compute_average(data, num_parties, values_per_party)
        end_time = time.time()  # Record end time

        runtime = end_time - start_time
        average_runtimes.append(runtime)
        results.append((average, runtime))

    average_runtime = sum(average_runtimes) / num_experiments


    print(f"Results for n = {num_parties} (No Privacy Protection SMPC):")
    for i, (result, runtime) in enumerate(results):
        print(f"Experiment {i + 1} - Runtime: {runtime} seconds")
        print(f"Result: {result}")
    print(f"Average Runtime for n = {num_parties}: {average_runtime} seconds\n")
    print()

    return average_runtime

In [ ]:
def shamir_secret_sharing(num_parties, values_per_party, num_experiments, secret, random_values):
    def share_secret(secret, num_parties, threshold):
        prime_num = 82171  # a large prime number
        coefficients = [secret] + [random.randint(1, prime_num - 1) for _ in range(threshold - 1)]
        x = symbols('x')
        polynomial = sum(coeff * x**i for i, coeff in enumerate(coefficients)) % prime_num
        points = [(i, polynomial.subs(x, i)) for i in range(1, num_parties + 1)]
        return points

    def interpolate(points, num_parties):
        x, y = symbols('x y')
        lagrange_basis = 0
        for xi, yi in points:
            basis = yi
            for xj, _ in points:
                if xj != xi:
                    basis *= (x - xj) / (xi - xj)
            lagrange_basis += basis


        # Create a SymPy expression from the computed Lagrange basis
        result = Poly(lagrange_basis, x).as_expr()

        return result

    total_runtime = 0
    results = []

    for _ in range(num_experiments):
        start_time = time.time()

        # Sharing the secret
        shares = share_secret(secret, num_parties, num_parties // 2)

        # Combining shares
        result = interpolate(shares[:num_parties // 2], num_parties)

        end_time = time.time()
        runtime = end_time - start_time
        total_runtime += runtime

        results.append((result, runtime))

    average_runtime = total_runtime / num_experiments

    print(f"Results for n = {num_parties} (Shamir's Secret Sharing SMPC):")
    for i, (result, runtime) in enumerate(results):
        print(f"Experiment {i + 1} - Runtime: {runtime} seconds")
        print(f"Result: {result}")
    print(f"Average Runtime for n = {num_parties}: {average_runtime} seconds\n")

    return average_runtime

In [ ]:
def paillier_encryption(num_parties, values_per_experiment, num_experiments, random_values):
    def generate_paillier_keypair():
        public_key, private_key = paillier.generate_paillier_keypair()
        return public_key, private_key

    def paillier_encrypt(public_key, plaintext):
        encrypted_value = public_key.encrypt(plaintext)
        return encrypted_value

    def paillier_decrypt(private_key, ciphertext):
        decrypted_value = private_key.decrypt(ciphertext)
        return decrypted_value


    runtimes = []

    total_runtime = 0
    experiment_results = []

    for i in range(num_experiments):
        integer_values = random_values[i]

        start_time = time.time()

        public_key, private_key = generate_paillier_keypair()
        encrypted_values = [paillier_encrypt(public_key, val) for val in integer_values]
        sum_encrypted = sum(encrypted_values)
        average_encrypted = paillier_decrypt(private_key, sum_encrypted)

        end_time = time.time()
        runtime = end_time - start_time
        total_runtime += runtime

        experiment_results.append((average_encrypted, runtime))

    average_runtime = total_runtime / num_experiments
    runtimes.append(average_runtime)

    print(f"Results for n = {num_parties} (Paillier Encryption SMPC):")
    for i, (result, runtime) in enumerate(experiment_results):
          print(f"Experiment {i + 1} - Runtime: {runtime} seconds")
          print(f"Result: {result}")
    print(f"Average Runtime for n = {num_parties}: {average_runtime} seconds\n")
    return average_runtime

In [ ]:
values_of_n = [10, 30, 50, 70, 100]

def main():
    num_experiments = 5
    values_per_party = 5

    no_privacy_runtimes = []
    shamir_runtimes = []
    paillier_runtimes = []

    for n in values_of_n:
        num_parties = n

        random_values = generate_random_values(num_parties, values_per_party, num_experiments)

        print(f"Running experiments for n = {num_parties} with No Privacy Protection SMPC:")
        runtime = no_privacy_protection(num_parties, values_per_party, num_experiments, random_values)
        no_privacy_runtimes.append(runtime)

        print(f"Running experiments for n = {num_parties} with Shamir's Secret Sharing SMPC:")
        secret = 55
        runtime = shamir_secret_sharing(num_parties, values_per_party, num_experiments, secret, random_values)
        shamir_runtimes.append(runtime)

        print(f"Running experiments for n = {num_parties} with Paillier Encryption SMPC:")
        runtime = paillier_encryption(num_parties, values_per_party, num_experiments,random_values)
        paillier_runtimes.append(runtime)

    return no_privacy_runtimes, shamir_runtimes, paillier_runtimes

if __name__ == "__main__":
    no_privacy_runtimes, shamir_runtimes, paillier_runtimes = main()

    fig_no_privacy = px.line(x=values_of_n, y=no_privacy_runtimes, labels={'x': 'n', 'y': 'Runtime (seconds)'},
                            title='No Privacy Protection SMPC')

    fig_shamir = px.line(x=values_of_n, y=shamir_runtimes, labels={'x': 'n', 'y': 'Runtime (seconds)'},
                         title="Shamir's Secret Sharing SMPC")

    fig_paillier = px.line(x=values_of_n, y=paillier_runtimes, labels={'x': 'n', 'y': 'Runtime (seconds)'},
                           title='Paillier Encryption SMPC')


    fig_combined = make_subplots(rows=1, cols=1)
    fig_combined.add_trace(go.Scatter(x=values_of_n, y=no_privacy_runtimes, mode='lines', name='No Privacy'))
    fig_combined.add_trace(go.Scatter(x=values_of_n, y=shamir_runtimes, mode='lines', name="Shamir's Secret Sharing"))
    fig_combined.add_trace(go.Scatter(x=values_of_n, y=paillier_runtimes, mode='lines', name='Paillier Encryption'))

    fig_combined.update_layout(
        title='Combined Runtimes for Different SMPC Approaches',
        xaxis_title='n',
        yaxis_title='Runtime (seconds)',)

    # Show the individual plots and the combined plot
    fig_no_privacy.show()
    fig_shamir.show()
    fig_paillier.show()
    fig_combined.show()
